In [1]:
from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, Normalizer
import pandas as pd
import tensorflow as tf
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import numpy as np

drive.mount("content/")

Drive already mounted at content/; to attempt to forcibly remount, call drive.mount("content/", force_remount=True).


In [2]:
drive.mount("content/")

Drive already mounted at content/; to attempt to forcibly remount, call drive.mount("content/", force_remount=True).


In [3]:
df = pd.read_csv("content/MyDrive/Colab Notebooks/data/HR-Employee-Attrition.csv")
df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,2,Female,94,3,2,Sales Executive,4,Single,5993,19479,8,Y,Yes,11,3,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,3,Male,61,2,2,Research Scientist,2,Married,5130,24907,1,Y,No,23,4,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,4,Male,92,2,1,Laboratory Technician,3,Single,2090,2396,6,Y,Yes,15,3,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,4,Female,56,3,1,Research Scientist,3,Married,2909,23159,1,Y,Yes,11,3,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,1,Male,40,3,1,Laboratory Technician,2,Married,3468,16632,9,Y,No,12,3,4,80,1,6,3,3,2,2,2,2


In [4]:
CATEGORICAL_FEATURES = [
                       "BusinessTravel",
                       "Department",
                       "EducationField",
                       "Gender",
                       "JobRole",
                       "MaritalStatus",
                       "Over18",
                       "OverTime",
]

In [5]:
df["Attrition"].value_counts()

No     1233
Yes     237
Name: Attrition, dtype: int64

In [6]:
y = np.array([1 if value == "Yes" else 0 for value in df["Attrition"].values])
X = df.drop("Attrition", axis=1)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [8]:
preprocessing_pipeline = make_pipeline(
  ColumnTransformer(
      [("ohe", OneHotEncoder(), CATEGORICAL_FEATURES)],
      remainder=StandardScaler()
    ),
    Normalizer()
)


In [9]:
model_pipline = make_pipeline(
    preprocessing_pipeline,
    RandomForestClassifier()
)

In [10]:
model_pipline.fit(X_train, y_train)

Pipeline(steps=[('pipeline',
                 Pipeline(steps=[('columntransformer',
                                  ColumnTransformer(remainder=StandardScaler(),
                                                    transformers=[('ohe',
                                                                   OneHotEncoder(),
                                                                   ['BusinessTravel',
                                                                    'Department',
                                                                    'EducationField',
                                                                    'Gender',
                                                                    'JobRole',
                                                                    'MaritalStatus',
                                                                    'Over18',
                                                                    'OverTime'])])),
                   

In [11]:
y_test_pred = model_pipline.predict(X_test)

In [12]:
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.88      1.00      0.94       321
           1       1.00      0.11      0.19        47

    accuracy                           0.89       368
   macro avg       0.94      0.55      0.57       368
weighted avg       0.90      0.89      0.84       368



In [13]:
preprocessing_pipeline.fit(X_train)
X_train_transformed = preprocessing_pipeline.transform(X_train)
X_test_transformed = preprocessing_pipeline.transform(X_test)

In [14]:
# Define the model - deep neural net
number_input_features = X_train_transformed.shape[1]
hidden_nodes_layer1 =  5
hidden_nodes_layer2 = 2

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 5)                 280       
                                                                 
 dense_1 (Dense)             (None, 2)                 12        
                                                                 
 dense_2 (Dense)             (None, 1)                 3         
                                                                 
Total params: 295
Trainable params: 295
Non-trainable params: 0
_________________________________________________________________


In [15]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [16]:
# Train the model
fit_model = nn.fit(X_train_transformed, y_train, epochs=1000)

Epoch 1/1000
35/35 [==============================] - 1s 4ms/step - loss: 0.7219 - accuracy: 0.2532
Epoch 2/1000
35/35 [==============================] - 0s 3ms/step - loss: 0.6888 - accuracy: 0.6316
Epoch 3/1000
35/35 [==============================] - 0s 3ms/step - loss: 0.6707 - accuracy: 0.8131
Epoch 4/1000
35/35 [==============================] - 0s 3ms/step - loss: 0.6587 - accuracy: 0.8267
Epoch 5/1000
35/35 [==============================] - 0s 3ms/step - loss: 0.6485 - accuracy: 0.8276
Epoch 6/1000
35/35 [==============================] - 0s 3ms/step - loss: 0.6392 - accuracy: 0.8276
Epoch 7/1000
35/35 [==============================] - 0s 3ms/step - loss: 0.6304 - accuracy: 0.8276
Epoch 8/1000
35/35 [==============================] - 0s 5ms/step - loss: 0.6221 - accuracy: 0.8276
Epoch 9/1000
35/35 [==============================] - 0s 5ms/step - loss: 0.6139 - accuracy: 0.8276
Epoch 10/1000
35/35 [==============================] - 0s 6ms/step - loss: 0.6062 - accuracy: 0.8276

In [17]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_transformed, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

12/12 - 0s - loss: 0.4592 - accuracy: 0.8668 - 165ms/epoch - 14ms/step
Loss: 0.4592464566230774, Accuracy: 0.866847813129425


In [18]:
y_test_pred_proba = nn.predict(X_test_transformed)

In [19]:
y_test_pred_nn = [1 if value else 0 for value in y_test_pred_proba > .5]

In [20]:
print(classification_report(y_test, y_test_pred_nn))

              precision    recall  f1-score   support

           0       0.90      0.95      0.93       321
           1       0.47      0.32      0.38        47

    accuracy                           0.87       368
   macro avg       0.69      0.63      0.65       368
weighted avg       0.85      0.87      0.86       368

